## Prepare Data

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
def fetch_data(classes):
  X = []
  Y = []
  for c in classes:
    path = os.path.join('/content/dataset', c)
    filenames = [os.path.join(path, name) for name in os.listdir(path)]
    X += filenames
    Y += [c, ] * len(filenames)
  return pd.DataFrame(zip(X,Y), columns=['filename', 'class'])

In [ ]:
df = fetch_data(['with_mask', 'without_mask'])
df.head()

In [ ]:
df = shuffle(df)
df.reset_index(inplace=True, drop=True)

In [ ]:
fig, ax = plt.subplots(4, 3)
fig.set_size_inches(15, 20)

for i in range(12):
  img = cv2.imread(df.iloc[i]['filename'])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (224, 224))
  ax[i//3, i%3].imshow(img)
  ax[i//3, i%3].set_title(df.iloc[i]['class'])

fig.tight_layout()

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.15)
train_df, val_df = train_test_split(train_df, test_size=0.15)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=(0.9, 1.3),
    zoom_range=0.1,
    horizontal_flip=True,
    rotation_range=10,
    rescale=1.0/255
)
val_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)

In [ ]:
BATCH_SIZE = 64
SIZE = 224

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    target_size=(SIZE,SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True,
    seed=12
)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    class_mode='binary',
    target_size=(SIZE,SIZE),
    batch_size=BATCH_SIZE
)
test_generator = val_datagen.flow_from_dataframe(
    dataframe=test_df,
    class_mode='binary',
    target_size=(SIZE,SIZE),
    batch_size=BATCH_SIZE
)

## Build model

In [ ]:
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Dense, Flatten, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Model

In [ ]:
def build_model(_input):
  x = Conv2D(64, (3,3), padding='same', activation='relu')(_input)
  x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
  x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
  x = AveragePooling2D(pool_size=(2,2), strides=(2,2))(x)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)
  
  x = Conv2D(128, (3,3), padding='same', activation='relu')(x)
  x = Conv2D(128, (3,3), padding='same', activation='relu')(x)
  x = Conv2D(128, (3,3), padding='same', activation='relu')(x)
  x = AveragePooling2D(pool_size=(2,2), strides=(2,2))(x)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)

  x = Conv2D(256, (3,3), padding='same', activation='relu')(x)
  x = Conv2D(256, (3,3), padding='same', activation='relu')(x)
  # x = Conv2D(256, (3,3), padding='same', activation='relu')(x)
  x = AveragePooling2D(pool_size=(2,2), strides=(2,2))(x)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)

  x = Conv2D(512, (3,3), padding='same', activation='relu')(x)
  x = Conv2D(512, (3,3), padding='same', activation='relu')(x)
  # x = Conv2D(512, (3,3), padding='same', activation='relu')(x)
  x = AveragePooling2D(pool_size=(2,2), strides=(2,2))(x)
  x = BatchNormalization()(x)
  x = Dropout(0.15)(x)

  x = Conv2D(512, (3,3), padding='same', activation='relu')(x)
  x = Conv2D(512, (3,3), padding='same', activation='relu')(x)
  # x = Conv2D(512, (3,3), padding='same', activation='relu')(x)
  x = AveragePooling2D(pool_size=(2,2), strides=(2,2))(x)
  x = BatchNormalization()(x)
  x = Dropout(0.1)(x)

  x = Flatten()(x)
  x = Dense(512, activation='relu')(x)
  x = Dense(1, activation='sigmoid')(x)

  return x

In [ ]:
_input = Input((SIZE,SIZE,3))
output = build_model(_input)

model = Model(_input, output)

model.summary()

## Training

In [ ]:
losses = []
val_losses = []

accuracies = []
val_accuracies = []

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
hist = model.fit(train_generator, batch_size=BATCH_SIZE, epochs=10, verbose=1, validation_data=val_generator)

In [ ]:
losses += hist.history['loss']
val_losses += hist.history['val_loss']

accuracies += hist.history['accuracy']
val_accuracies += hist.history['val_accuracy']


In [ ]:
plt.plot(losses, 'r-', label='loss')
plt.plot(val_losses, 'b-', label='val_loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(accuracies, 'm--', label='accuracy')
plt.plot(val_accuracies, 'g--', label='val_accuracy')
plt.legend()

## Eveluate

In [ ]:
def get_accuracy(pred):
  pred = pred[0][0]
  if pred > 0.5:
    return int(pred * 100)
  return int((1-pred) * 100)

In [ ]:
ret = model.evaluate(test_generator, batch_size=64)

In [ ]:
batch_img, batch_label = next(test_generator)

In [ ]:
fig, ax = plt.subplots(4, 3)
fig.set_size_inches(15, 20)
labels_encoder = ['with_mash', 'without_mask']

poses = np.random.choice(range(64), 12, replace=False)
for i in range(12):
  img, gt = batch_img[poses[i]], batch_label[poses[i]]
  pred = model.predict(img[np.newaxis, ...])
  img = (img * 255).astype(np.uint8)
  # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  ax[i//3, i%3].imshow(img)
  title = 'Ground truth: {0}\nPredict: {1}   {2}%'.format(labels_encoder[int(gt)], labels_encoder[int(pred)], get_accuracy(pred))
  ax[i//3, i%3].set_title(title)
  

fig.tight_layout()

#### Test with arbitrary image

In [ ]:
!wget https://vinmec-prod.s3.amazonaws.com/images/20200203_093825_470517_unnamed.max-1800x1800.jpg -O image.jpg
!wget http://benhvienthanhvubaclieu.com/cms/static/site/sale_medicbaclieu/uploads/ckeditor/images.thumb.d06d5e1c-4cb4-4828-b607-e555aa58a618.jpg -O image1.jpg

In [ ]:
img = cv2.imread('image.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224, 224))
img = img / 255.0


In [ ]:
pred = model.predict(img[np.newaxis, ...])

In [ ]:
plt.imshow(img)
title = '{0} {1}%'.format(labels_encoder[int(pred)], get_accuracy(pred))
plt.title(title)